In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
dataset=pd.read_csv('Churn_Modelling.csv')

#dependent and independent
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]


In [3]:
# applying encoding for geography and gender
geography = pd.get_dummies(X['Geography'],drop_first=True)
gender = pd.get_dummies(X['Gender'],drop_first=True)

# concatination
X=pd.concat([X,geography,gender],axis=1)
X=X.drop(['Geography','Gender'],axis=1)


In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

<ipython-input-5-7bb571729a7c>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [6]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        #here hp is parameter when we are are running randomsearch and we are saying neural network can have layers from 2 to 20 (hidden layers)
        model.add(layers.Dense(units=hp.Int('unit_'+ str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
        #number of neurons ranges from 32 to 512
    model.add(layers.Dense(1,activation='sigmoid'))
    #since this is regression problem so output will be only 1
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',[.001,.0001,.00001])),
            loss='binary_crossentropy',
        metrics=['accuracy'])
    return model                  
        

In [7]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='keras-tuner_ann')

In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
unit_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
unit_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05], 'ordered': True}


In [9]:
tuner.search(X_train, y_train,
             epochs=50,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 02m 22s]
val_accuracy: 0.8618333339691162

Best val_accuracy So Far: 0.8658333222071329
Total elapsed time: 00h 33m 13s
INFO:tensorflow:Oracle triggered exit


In [10]:
tuner.results_summary()

Results summary
Results in keras-tuner_ann\untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 18
unit_0: 448
unit_1: 224
learning_rate: 0.0001
unit_2: 32
unit_3: 32
unit_4: 32
unit_5: 32
unit_6: 32
unit_7: 32
unit_8: 32
unit_9: 32
unit_10: 32
unit_11: 32
unit_12: 32
unit_13: 32
unit_14: 32
unit_15: 32
unit_16: 32
unit_17: 32
Score: 0.8658333222071329
Trial summary
Hyperparameters:
num_layers: 15
unit_0: 416
unit_1: 96
learning_rate: 0.0001
unit_2: 448
unit_3: 480
unit_4: 160
unit_5: 320
unit_6: 384
unit_7: 416
unit_8: 352
unit_9: 480
unit_10: 416
unit_11: 352
unit_12: 128
unit_13: 384
unit_14: 384
unit_15: 448
unit_16: 416
unit_17: 320
Score: 0.8636666735013326
Trial summary
Hyperparameters:
num_layers: 12
unit_0: 96
unit_1: 480
learning_rate: 1e-05
unit_2: 160
unit_3: 512
unit_4: 320
unit_5: 352
unit_6: 64
unit_7: 160
unit_8: 192
unit_9: 224
unit_10: 128
unit_11: 96
unit_12: 192
unit_13: 128
unit_14: 288
u